In [1]:
import util.Landau_Potential_Diffusion as Landau
import util.curvature as curve

import xarray as xr
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime
from skimage import measure
from scipy import ndimage
import scipy.integrate as sin
from scipy.optimize import curve_fit

import pickle
import sys

In [3]:
path = '/nfs/twcroninlab002/tbeucler/ERA5/ERA5_LHF_SHF/'

In [4]:
PRECIP = xr.open_mfdataset(path+'????/??PRECIP.nc',combine='by_coords')

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [6]:
dt = 3600
tcoor = dt*np.arange(0,PRECIP.time.shape[0])

In [9]:
latmin = -30
latmax = 30
lonmin = 300
lonmax = 360

lonAtl = PRECIP.longitude.sel({'longitude':slice(lonmin,lonmax)})
latAtl = PRECIP.latitude.sel({'latitude':slice(latmax,latmin)})

In [10]:
PRECAtl = 1e3*PRECIP['tp'].sel({'longitude':slice(lonmin,lonmax),'latitude':slice(latmax,latmin)}) # In mm/day or kg/m2/day

In [11]:
YEAR = np.linspace(2000,2018,19).astype(int)
print(YEAR)

[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018]


In [20]:
hf = open('PKL_DATA/10_13_Pp999.pkl','rb')
tmp = pickle.load(hf)
meanP = tmp['Pp999mean']

In [21]:
P999 = {}
it_tot = 0
for i,year in enumerate(YEAR):
    print('i=',i,' & year=',year)
    date1 = str(year)+'-01-01T00:00:00'
    date2 = str(year)+'-12-31T23:00:00'
    Nt = PRECAtl.time.sel({'time':slice(date1,date2)}).shape[0]
    P999[str(year)] = np.zeros((Nt,))
    for it in range(Nt):
        print('it=',it,' & it_tot=',it_tot,'               ',end='\r')
        P999[str(year)][it] = np.sum(PRECAtl[it_tot,:,:].values>meanP)
        it_tot+=1

i= 0  & year= 2000


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-21-2865f4829aad>", line 11, in <module>
    P999[str(year)][it] = np.sum(PRECAtl[it_tot,:,:].values>meanP)
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/xarray/core/dataarray.py", line 488, in values
    return self.variable.values
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/xarray/core/variable.py", line 389, in values
    return _as_array_or_item(self._data)
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/xarray/core/variable.py", line 210, in _as_array_or_item
    data = np.asarray(data)
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/numpy/core/numeric.py", line 538, in asarray
    return array(a, dtype, copy=False, order=order)
  File "/home/tbeucler/miniconda3/lib/python3.7/site-packages/das

KeyboardInterrupt: 

In [23]:
hf = open('PKL_DATA/10_13_P999.pkl','wb')
P_data = {"P999":P999,"Pp999mean":meanP}
pickle.dump(P_data,hf)
hf.close()